# Write a program on Generative AI that perform a following task
 1. take a problem statement from user
 2. create a short story/ news/ writer / research article etc (base on the mention task by the user)
 3. also create set of images to support step 2
 4. creates visual (audio + video) using steps 2 and 3

In [ ]:
!pip install torch transformers diffusers accelerate moviepy soundfile
!pip install pywhatkit python-dotenv beautifulsoup4 rich groq requests keyboard wikipedia

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from bs4 import BeautifulSoup
from rich import print
from groq import Groq
import os
import webbrowser
import subprocess
import requests
import keyboard
import asyncio
import wikipedia
import torch
from diffusers import StableDiffusionPipeline
from transformers import pipeline, BarkModel, AutoProcessor
from moviepy.editor import ImageSequenceClip, AudioFileClip, concatenate_videoclips
from scipy.io.wavfile import write as write_wav
import numpy as np

# --- Configuration ---
# Replace this placeholder with your actual Groq API key
GROQ_API_KEY = "your_API_Key"
groq_client = Groq(api_key=GROQ_API_KEY)

# --- Hugging Face API Key for Image/Audio Models (if needed) ---
# For some models, you might need a Hugging Face API key.
# For simplicity and to avoid over-complicating this example,
# we'll assume the public models used here don't require one,
# but it's good practice to be aware of this.
HUGGINGFACE_API_KEY = "your_API_Key"

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
OUTPUT_DIR = "generated_content"
os.makedirs(OUTPUT_DIR, exist_ok=True)


# --- Text Generation Function ---
def contentWrite(query):
    try:
        response = groq_client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": query}],
            temperature=0.7,
            max_tokens=1000,
            top_p=0.95,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        result_content = response.choices[0].message.content.strip()

        if not result_content:
            return "No content generated."

        print(result_content)
        return result_content

    except Exception as e:
        return f"Error: {e}"


# --- Wikipedia Summary Function ---
def WikiSummary(topic):
    try:
        summary = wikipedia.summary(topic, sentences=2)
        print(f"[bold magenta]Wikipedia Summary:[/bold magenta] {summary}")
        return summary
    except Exception as e:
        print(f"[bold red]Wikipedia Error:[/bold red] {e}")
        return f"Error: {e}"


# --- New Functions for Image, Audio, and Video Generation ---
def generate_images_from_text(text: str) -> list:
    """Generates a series of images from key sentences in the text."""
    sentences = [s.strip() for s in text.split('.') if s.strip()]

    # Initialize the Stable Diffusion pipeline
    pipe = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16 if device == 'cuda' else torch.float32
    ).to(device)

    image_paths = []
    print("\n--- Generating Images ---")
    for i, sentence in enumerate(sentences):
        # Limit to 4 images for a quick demo to save time and resources
        if i >= 4:
            break

        # Create a detailed prompt for better image quality
        prompt = f"Cinematic still of a scene from a story: {sentence}, highly detailed, professional photography, 8k"

        # Generate the image
        image = pipe(prompt).images[0]

        # Save the image to the output directory
        image_path = os.path.join(OUTPUT_DIR, f"image_{i}.png")
        image.save(image_path)
        image_paths.append(image_path)
        print(f"Saved image {i+1} to {image_path}")

    return image_paths

def generate_audio_from_text(text: str) -> str:
    """Converts text into a speech audio file using a text-to-speech model."""
    try:
        processor = AutoProcessor.from_pretrained("suno/bark")
        model = BarkModel.from_pretrained("suno/bark")
        if device == 'cuda':
            model = model.to(device)

        voice_preset = "v2/en_speaker_6"
        inputs = processor(text, voice_preset=voice_preset)

        with torch.no_grad():
            audio_array = model.generate(**inputs, do_sample=True)

        audio_path = os.path.join(OUTPUT_DIR, "audio.wav")
        # Ensure audio data is a single channel for a standard .wav file
        audio_data = audio_array.cpu().numpy().squeeze()
        write_wav(audio_path, model.generation_config.sample_rate, audio_data)

        print("\n--- Audio Generated ---")
        print(f"Saved audio to {audio_path}")
        return audio_path
    except Exception as e:
        print(f"Audio generation failed: {e}")
        return None

def create_video_from_assets(image_paths: list, audio_path: str) -> str:
    """Combines images and an audio file into a video."""
    if not image_paths or not audio_path:
        print("Video creation skipped due to missing images or audio.")
        return ""

    try:
        audio_clip = AudioFileClip(audio_path)
        duration_per_image = audio_clip.duration / len(image_paths)

        # Create image clips, each with a duration that matches a segment of the audio
        image_clips = [ImageSequenceClip([path], durations=[duration_per_image]) for path in image_paths]

        # Concatenate image clips into a single video clip
        video_clip = concatenate_videoclips(image_clips)

        # Set the audio of the video clip
        final_video_clip = video_clip.set_audio(audio_clip)

        video_path = os.path.join(OUTPUT_DIR, "final_video.mp4")
        final_video_clip.write_videofile(video_path, fps=24, codec='libx264')

        print("\n--- Video Created ---")
        print(f"Final video saved to {video_path}")
        return video_path
    except Exception as e:
        print(f"Video creation failed: {e}")
        return None


# --- Main Program Execution ---
if __name__ == "__main__":
    prompt = input("Problem statement: ")
    prompt_type = input("Enter a type (story/ news/ writer / research /article etc): ")

    # 1. Generate text content
    content = contentWrite(f"write a {prompt_type} on {prompt}")

    # 2. Add Wikipedia Summary if it's a 'research' type
    if prompt_type == "research":
        WikiSummary(prompt)

    # 3. Check if content was generated before proceeding
    if content and content != "No content generated.":
        # 4. Generate images from the content
        image_files = generate_images_from_text(content)

        # 5. Generate audio from the content
        audio_file = generate_audio_from_text(content)

        # 6. Create the final video
        create_video_from_assets(image_files, audio_file)

Problem statement: bloody mary
Enter a type (story/ news/ writer / research /article etc): story


It was a dark and stormy night, the perfect setting for a gathering of friends to share a spooky tale or two. Emma,
a history buff, had just moved into a new town and was eager to make some friends. She had heard about a legendary 
game that was popular among the local high school crowd, and she was determined to try it out.

The game was called "Bloody Mary," and the rules were simple: gather a group of friends, light a candle, and 
whisper the name "Mary" three times while looking into a mirror. According to legend, Mary was a queen who had been
beheaded for her crimes, and her spirit was said to appear in the mirror, bringing death and destruction to those 
who summoned her.

Emma was skeptical at first, but her friends were convinced that the game was real, and they were determined to 
prove it to her. They gathered in Emma's new home, a large old house on the outskirts of town, and set up the 
candle and mirror on a table in the living room.

As they lit the candle and began to whisper the name "Mary," a strange feeling crept over the room. The air seemed 
to grow colder, and the shadows on the walls began to twist and writhe like living things. Emma felt a shiver run 
down her spine as she looked into the mirror, but her friends were undeterred, their eyes gleaming with excitement.

The third time they whispered the name, a sudden gust of wind blew out the candle, plunging the room into darkness.
Emma's friends gasped as a figure began to take shape in the mirror, a figure with a crown on her head and a look 
of outrage in her eyes.

"Who has summoned me?" the figure demanded, her voice like a crackling flame. "Who has disturbed the rest of the 
dead?"

Emma tried to speak, but her voice was caught in her throat. Her friends, however, were quick to respond, their 
voices trembling with fear.

"We didn't mean to, Mary," one of them stammered. "We just wanted to prove the game was real."

The figure in the mirror glared at them, her eyes blazing with anger. "You have disturbed the rest of the dead," 
she repeated, her voice growing louder and more menacing. "You will pay for your foolishness."

As the figure began to move closer to the mirror, the room was filled with an unearthly light. Emma and her friends
cowered in terror, unable to move or speak. They were trapped in a living hell, and they knew they would never be 
able to escape.

Just as the figure was about to reach out and touch the mirror, the power in the house suddenly went out, plunging 
the room into total darkness. Emma and her friends were left to sit in the darkness, their hearts racing with fear,
and their minds filled with the knowledge that they had unleashed a terrible evil upon themselves.

As they sat in the darkness, they began to hear strange noises, the creaking of wooden floorboards, the rustling of
fabric, and the sound of footsteps, as if someone - or something - was moving around the house, searching for them.
They knew that they had to get out of the house, but their bodies seemed to be paralyzed with fear, and they were 
unable to move.

As the night wore on, the noises grew louder and more intense, and Emma and her friends began to realize that they 
were not alone in the house. They were being stalked by the spirit of Bloody Mary, and they knew that they would 
never be able to escape her wrath.

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

--- Generating Images ---

  0%|          | 0/50 [00:00<?, ?it/s]

...

Moviepy - Building video generated_content/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video generated_content/final_video.mp4



Moviepy - Done !
Moviepy - video ready generated_content/final_video.mp4


--- Video Created ---

Final video saved to generated_content/final_video.mp4